Microsoft AI Rangers Demo
# Large Scale Image Classification
__X-Ray Classification using CheXpert__

<img src="images/dp-chexpert.png" width=800 />

### Goal
The goal of this notebook is demonstrate feasiblity of large scale multi-label image classification from radiographs using Automated Machine Learning. For this, we are using [CheXpert:](https://stanfordmlgroup.github.io/competitions/chexpert/) a large chest X-Ray dataset. Given that the original size of CheXpert is ~400GB, we will use the small version (image resized) which is ~11GB.


### Steps
1. Upload data (small dataset) to the cloud
2. Convert the data to JSONL
3. Set AutoML Run
4. Configure parameters and run

At the end, we show results using CheXpert standard image size (no resizing).

This notebook was developed and tested using an Azure ML STANDARD_D13_V2 CPU [compute instance](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-create-manage-compute-instance?tabs=python) and Python [SDK azureml v1](https://learn.microsoft.com/en-us/python/api/overview/azure/ml/?view=azure-ml-py&preserve-view=true).


# 1. Upload data to the cloud

## Download and extracting CheXpert
The dataset is available for download [here](https://stanfordmlgroup.github.io/competitions/chexpert/). We have downloaded the dataset from above url and zipped the images files into the following structure:

<img src="images/dp-chexpert-file-structure.png" width=200 />


In [ ]:
# remove files from /tmp to avoid potential overlap from previous runs
!rm /tmp/CheXpert-v1.0-small.zip
!rm -r /tmp/CheXpert-v1.0-small

# Download the small version of the dataset
!wget "LINK_TO_FILE" -P /tmp
!unzip -q /tmp/CheXpert-v1.0-small.zip -d /tmp

In [29]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
ds = ws.get_default_datastore()

chexpert_local_path = '/tmp/CheXpert-v1.0-small/'
blob_chexpert_target_name = "chexpert_v1_small/"

# Upload image files to livecell folder in AML datastore
ds.upload(src_dir=chexpert_local_path, target_path=blob_chexpert_target_name, overwrite=True, show_progress=False)


## 2. Convert the data to JSONL

In [32]:
import pandas as pd
import json, os

df_train = pd.read_csv( chexpert_local_path + 'train.csv')
df_train = df_train[df_train['Frontal/Lateral'] == 'Frontal']  

df_valid = pd.read_csv(chexpert_local_path + 'valid.csv')
df_valid = df_valid[df_valid['Frontal/Lateral'] == 'Frontal']  

print(df_train.shape)
df_train.head(4)

(191027, 19)


,Path,Sex,Age,Frontal/Lateral,AP/PA,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,Lung Lesion,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
0,CheXpert-v1.0-small/train/patient00001/study1/...,Female,68,Frontal,AP,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,1.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,Female,87,Frontal,AP,NaN,NaN,-1.0,1.0,NaN,-1.0,-1.0,NaN,-1.0,NaN,-1.0,NaN,1.0,NaN
2,CheXpert-v1.0-small/train/patient00002/study1/...,Female,83,Frontal,AP,NaN,NaN,NaN,1.0,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN
4,CheXpert-v1.0-small/train/patient00003/study1/...,Male,41,Frontal,AP,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [33]:
pathologies = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']
# sample json line dictionary
json_line_sample = {
    "image_url": "AmlDatastore://",
    "label": ""
    }

chexpert_dataset_name = "CheXpert-v1.0-small"
# To process standard CheXpert, just remove the '-small' characters:
# chexpert_dataset_name = "CheXpert-v1.0"

label_dir = 'label_files/'
labels_chexpert_local_path = chexpert_local_path + label_dir

if not os.path.exists(labels_chexpert_local_path):
    os.mkdir(labels_chexpert_local_path)

chexpert_train_jsonl_file_name = "chexpert_train.jsonl"
chexpert_validation_jsonl_file_name = "chexpert_validation.jsonl"

# Path to the training and validation files
train_annotations_file = labels_chexpert_local_path + chexpert_train_jsonl_file_name
validation_annotations_file = labels_chexpert_local_path + chexpert_validation_jsonl_file_name

with open(train_annotations_file, "w") as train_f:
    for idx, row in df_train.iterrows():
        pathology_list = [pathology for pathology in pathologies if row[pathology] == 1]
        if len(pathology_list) == 0:
            pathology_list.append("X_other")
        json_line = dict(json_line_sample)
        json_line["image_url"] += "workspaceblobstore/" + row.Path.replace(chexpert_dataset_name,blob_chexpert_target_name) 
        json_line["label"] = pathology_list  
        train_f.write(json.dumps(json_line) + "\n")

with open(validation_annotations_file, "w") as validation_f:
    for idx, row in df_valid.iterrows():
        pathology_list = [pathology for pathology in pathologies if row[pathology] == 1]
        if len(pathology_list) == 0:
            pathology_list.append("X_other")
        json_line = dict(json_line_sample)
        json_line["image_url"] += "workspaceblobstore/" +  row.Path.replace(chexpert_dataset_name,blob_chexpert_target_name) 
        json_line["label"] = pathology_list  
        validation_f.write(json.dumps(json_line) + "\n")


In [35]:
ds = ws.get_default_datastore()

labels_chexpert_blob_path = blob_chexpert_target_name + label_dir

# Upload json files to the label folder in AML datastore
ds.upload(src_dir=labels_chexpert_local_path, target_path=labels_chexpert_blob_path, overwrite=True, show_progress=True)

Uploading an estimated of 2 files
Uploading /tmp/CheXpert-v1.0-small/label_files/chexpert_validation.jsonl
Uploaded /tmp/CheXpert-v1.0-small/label_files/chexpert_validation.jsonl, 1 files out of an estimated total of 2
Uploading /tmp/CheXpert-v1.0-small/label_files/chexpert_train.jsonl
Uploaded /tmp/CheXpert-v1.0-small/label_files/chexpert_train.jsonl, 2 files out of an estimated total of 2
Uploaded 2 files


$AZUREML_DATAREFERENCE_fe2c2f936a684cb6b4cd658e0e6acaf3

In [37]:
from azureml.contrib.dataset.labeled_dataset import _LabeledDatasetFactory, LabeledDatasetTask
from azureml.core import Dataset

# Path to the training and validation files
train_dataset_name = "chexpert_train"
valid_dataset_name = "chexpert_valid"

# create training dataset
training_dataset = _LabeledDatasetFactory.from_json_lines(
    task=LabeledDatasetTask.IMAGE_MULTI_LABEL_CLASSIFICATION, path=ds.path(labels_chexpert_blob_path + chexpert_train_jsonl_file_name))
training_dataset = training_dataset.register(workspace=ws, name=train_dataset_name)

# create validation dataset
validation_dataset = _LabeledDatasetFactory.from_json_lines(
    task=LabeledDatasetTask.IMAGE_MULTI_LABEL_CLASSIFICATION, path=ds.path(labels_chexpert_blob_path + chexpert_validation_jsonl_file_name))
validation_dataset = validation_dataset.register(workspace=ws, name=valid_dataset_name)

print("Training dataset name: " + training_dataset.name)
print("Validation dataset name: " + validation_dataset.name)

Training dataset name: chexpert_train
Validation dataset name: chexpert_valid


In [40]:
validation_dataset.to_pandas_dataframe().head(10)

,image_url,label
0,StreamInfo(AmlDatastore://workspaceblobstore/c...,[Cardiomegaly]
1,StreamInfo(AmlDatastore://workspaceblobstore/c...,[X_other]
2,StreamInfo(AmlDatastore://workspaceblobstore/c...,[Edema]
3,StreamInfo(AmlDatastore://workspaceblobstore/c...,[X_other]
4,StreamInfo(AmlDatastore://workspaceblobstore/c...,"[Atelectasis, Pleural Effusion]"
5,StreamInfo(AmlDatastore://workspaceblobstore/c...,"[Cardiomegaly, Atelectasis]"
6,StreamInfo(AmlDatastore://workspaceblobstore/c...,[X_other]
7,StreamInfo(AmlDatastore://workspaceblobstore/c...,[X_other]
8,StreamInfo(AmlDatastore://workspaceblobstore/c...,"[Cardiomegaly, Consolidation, Atelectasis, Ple..."
9,StreamInfo(AmlDatastore://workspaceblobstore/c...,[Cardiomegaly]


## Compute target setup
You need to provide a [Compute Target](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#compute-target) that will be used for your AutoML model training. AutoML models for image tasks require GPU SKUs and support NC and ND families. We recommend using the NCsv3-series (with v100 GPUs) for faster training. Using a compute target with a multi-GPU VM SKU will leverage the multiple GPUs to speed up training. Additionally, setting up a compute target with multiple nodes will allow for faster model training by leveraging parallelism, when tuning hyperparameters for your model.

In [29]:
from azureml.core.compute import AmlCompute, ComputeTarget

cluster_name = "gpu-clu-nv24v3"

try:
    compute_target = ws.compute_targets[cluster_name]
    print('Found existing compute target.')
except KeyError:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6', 
                                                           vm_priority='lowpriority', # or 'dedicated
                                                           idle_seconds_before_scaledown=1800,
                                                           min_nodes=0, 
                                                           max_nodes=4)

    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)
    
# Can poll for a minimum number of nodes and for a specific timeout.
# If no min_node_count is provided, it will use the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating a new compute target...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [23]:
from azureml.core import Experiment

experiment_name = "automl-chexpert-classification-multilabel"
experiment = Experiment(ws, name=experiment_name)

We use the configuration below, using Vision Transformer (ViT) [vitb16r224](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-auto-train-image-models?tabs=cli#supported-model-algorithms) with image 256 image resize and 225 image center crop. To visualize performance for all epochs, we don't use an [early termination policy](https://learn.microsoft.com/en-us/azure/machine-learning/v1/how-to-auto-train-image-models-v1#early-termination-policies). In larger datasets and depending on the compute budget you can specify an early termination policy such as [Median stopping policy](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-tune-hyperparameters#median-stopping-policy).

In [31]:
from azureml.automl.core.shared.constants import ImageTask
from azureml.train.automl import AutoMLImageConfig
from azureml.train.hyperdrive import BanditPolicy, RandomParameterSampling,GridParameterSampling
from azureml.train.hyperdrive import choice, uniform

parameter_space = {
    "learning_rate": choice(0.0001, .0003, .0005),
    "early_stopping": choice(0),
    "weighted_loss": choice(1,2),
    "number_of_epochs": 10,
    "model": choice(
        {
            # model-specific, valid_resize_size should be larger or equal than valid_crop_size
            "model_name": choice("vitb16r224"),            
            "valid_resize_size": choice(256),
            "valid_crop_size": choice(224),  # model-specific
            "train_crop_size": choice(224),  # model-specific
        }
    ),
}

tuning_settings = {
    "iterations": 6,
    "max_concurrent_iterations": 4,
    "hyperparameter_sampling": GridParameterSampling(parameter_space),
    "early_termination_policy":None,
}

automl_image_config = AutoMLImageConfig(
    task=ImageTask.IMAGE_CLASSIFICATION_MULTILABEL,
    compute_target=compute_target,
    training_data=training_dataset,
    validation_data=validation_dataset,
    **tuning_settings,
    
)

In [ ]:
automl_image_run = experiment.submit(automl_image_config)

Visualize the different configurations that were tried using the HyperDrive UI.

In [24]:
from azureml.core import Run
hyperdrive_run = Run(experiment=experiment, run_id=automl_image_run.id + '_HD')
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl-chexpert-classification-multilabel,AutoML_b7ae46a4-9972-468c-9aeb-18709b09e7f7_HD,hyperdrive,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
from azureml.widgets import RunDetails

runID = "AutoML_b7ae46a4-9972-468c-9aeb-18709b09e7f7" 
run = [r for r in experiment.get_runs() if r.id == runID][0]
print(run.id)
RunDetails(run).show()

AutoML_b7ae46a4-9972-468c-9aeb-18709b09e7f7


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Download the best model

In [27]:
best_child_run = run.get_best_child()
model_name = best_child_run.properties['model_name']
model = best_child_run.register_model(model_name = model_name, model_path='outputs/model.pt')

# Results using 'small CheXpert' dataset

<img src="images/dp-chexpert_small-runs.png" width=800 />

<img src="images/dp-chexpert_small-runs_parallel_coordinates_chart.png" width=800 />

# Bonus

### Results using standard (non-resized) CheXpert dataset

We show results of training CheXpert using the standard image size. The two graphs below show overall performance using a 'serexnet' model. In this case, we performed an exaustive evaluation using by disabaling the early terminal policy (configuration parameters are below). 

You can reproduce results by just downloading/unziping standard CheXpert dataset using the steps above.

<img src="images/dp-chexpert-runs.png" width=800 >

<img src="images/dp-chexpert-runs_parallel_coordinates_chart.png" width=800 >

### Effect of [hyperparameters](https://learn.microsoft.com/en-us/azure/machine-learning/reference-automl-images-hyperparameters#image-classification-multi-class-and-multi-label-specific-hyperparameters)

We can easily visualize effects of hyperparameters. The two graphs below show effect of learning rate and weighted loss with respect to the AUC macro (metric). Based on the graphs below, we osbserve that performance (AUC Macro) increases for:

1. Larger learning rates (greater than 0.001) 
2. Weighted loss with sqrt.(class_weights), which corresponds to value 1 (walue 2 corresponds to weighted loss with class_weights).

<img src="images/dp-chexpert-scatter-AUC_lr.png" width=800 >

<img src="images/dp-chexpert-scatter-AUC_wl.png" width=800 >



## The parameter settings for the runs above:

In [ ]:
parameter_space = {
    "learning_rate": choice(0.0001, .0003, .0005),
    "early_stopping": choice(0),
    "weighted_loss": choice(1,2),
    "number_of_epochs": 40,
    "model": choice(
        {
            # model-specific, valid_resize_size should be larger or equal than valid_crop_size
            "model_name": choice("seresnext"),            
            "valid_resize_size": choice(512),
            "valid_crop_size": choice(488),  # model-specific
            "train_crop_size": choice(488),  # model-specific
            'training_batch_size': choice(48), 
            'validation_batch_size': choice(48)
        },
        {
            # model-specific, valid_resize_size should be larger or equal than valid_crop_size
            "model_name": choice("seresnext"),            
            "valid_resize_size": choice(288),
            "valid_crop_size": choice(224),  # model-specific
            "train_crop_size": choice(224),  # model-specific
            'training_batch_size': choice(48), 
            'validation_batch_size': choice(48),
        },
        {
            # model-specific, valid_resize_size should be larger or equal than valid_crop_size
            "model_name": choice("seresnext"),            
            "valid_resize_size": choice(320),
            "valid_crop_size": choice(224),  # model-specific
            "train_crop_size": choice(224),  # model-specific
            'training_batch_size': choice(48), 
            'validation_batch_size': choice(48),
        },
        {
            # model-specific, valid_resize_size should be larger or equal than valid_crop_size
            "model_name": choice("seresnext"),            
            "valid_resize_size": choice(352),
            "valid_crop_size": choice(256),  # model-specific
            "train_crop_size": choice(256),  # model-specific
            'training_batch_size': choice(48), 
            'validation_batch_size': choice(48),
        }
    ),
}

tuning_settings = {
    "iterations": 24,
    "max_concurrent_iterations": 8,
    "hyperparameter_sampling": GridParameterSampling(parameter_space),
    "early_termination_policy":None,
}


